# Testing the speed of various plotting methods

In the context of wavepy runtime plotting.

In [ ]:
import numpy as np                    # maths toolbox
import matplotlib.pyplot as plt       # plotting toolbox

from WavePyClasses import Grid, Model, HomogeneousModel, Source, Receiver, plot_field


## Random array plotting

In [ ]:
arr = np.random.randint(0,10, (15,15))
arr

In [ ]:
arr[::2,::2]

In [ ]:
gridje = Grid(200e3, 100e3, 400, 400)
arrbig = np.random.randint(0,10, gridje.X.shape)

In [ ]:
%timeit plot_field(gridje, arrbig, draw=True, verbose=False)

## Testing wavefield plotting

In [ ]:
import WaveProp as waveprop

### Set up simulation

In [ ]:
# Setting up the model domain:

Lx = 100e3 # horizontal length of the model in metres
Lz = 60e3 # vertical depth of the model in metres

dx = 400.  # horizontal grid spacing [m]
dz = 400.  # vertical grid spacing [m]

grid = Grid(Lx, Lz, dx, dz)

In [ ]:
# make model

rho = 2600 # density in kg/m^3
vp  = 5800 # P velocity in m/s
vs  = 3200 # S velocity in m/s

# make a model out of the supplied values
model = HomogeneousModel(grid, rho=rho, vp=vp, vs=vs)

In [ ]:
# make source/rec

src = Source(loc_x=40e3, loc_z=30e3) 
rec = Receiver(loc_x=60e3, loc_z=30e3)

# set src timing

src.set_time(t_max=8., dt=0.03)

src.set_stf(
    src_type='ricker',
    src_direction = (1,0),
    t_0 = 2.5,
    tau = 4,
)

print(src)
print(rec)

In [ ]:
# put all the absorbing boundary info into a Python dictionary

absorbing_boundaries = {
    'left': 25e3,
    'right': 25e3,
    'bottom': 25e3 
    }

### Run single simulation to get `fw_fields`

In [ ]:
%matplotlib notebook

receivers, fw_fields = waveprop.run_waveprop(
    src, rec, model, absorbing_boundaries, 
    plot_wavefield=True, plot_wavefield_every=40,
    store_forward_fields=True,
    verbose=True,
    half_grid=True, plot_contours=True,
    levels=20,
)

### Plot one fw_field in different ways

In [ ]:
src_amp = [np.linalg.norm((x,z)) for x,z in zip(src.stf.x, src.stf.z)]
cmaks = 1e-12*np.max(src_amp)

In [ ]:
%matplotlib inline

plt.close('all')
plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks);
plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, half_grid=True);
plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, plot_contours=True, levels=14, colorbar=True);
plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, plot_contours=True, levels=30, colorbar=True);
plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, plot_contours=True, levels=50, colorbar=True);


### Timing of a single fw_field plotting

matplotlib inline

In [ ]:
%matplotlib inline
%timeit plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks); plt.show(); plt.close('all')
%timeit plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, half_grid=True); plt.show(); plt.close('all')
%timeit plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, plot_contours=True, levels=14); plt.show(); plt.close('all')
%timeit plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, plot_contours=True, levels=30); plt.show(); plt.close('all')
%timeit plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, plot_contours=True, levels=50); plt.show(); plt.close('all')

#### Different levels of contours, with and without interpolating

They look the same to me.

In [ ]:
%matplotlib inline
print('normal')
plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, plot_contours=True, levels=14, colorbar=True); plt.show()
print('interp\'d')
plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, plot_contours=True, levels=14, colorbar=True, half_grid=True); plt.show()
print('normal')
plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, plot_contours=True, levels=30, colorbar=True); plt.show()
print('interp\'d')
plot_field(grid, fw_fields['ux_fw'][20], cmaks=cmaks, plot_contours=True, levels=30, colorbar=True, half_grid=True); plt.show()

### Timing of a full simulation for different plotting options

In [ ]:
%matplotlib notebook

#### Original

In [ ]:
%%timeit 
receivers, fw_fields = waveprop.run_waveprop(
    src, rec, model, absorbing_boundaries, 
    plot_wavefield=True, plot_wavefield_every=40,
    store_forward_fields=True,
    verbose=True,
    half_grid=False, plot_contours=False,
);
plt.show()

#### half grid

Looks kinda ugly.

In [ ]:
%%timeit 
receivers, fw_fields = waveprop.run_waveprop(
    src, rec, model, absorbing_boundaries, 
    plot_wavefield=True, plot_wavefield_every=40,
    store_forward_fields=True,
    verbose=True,
    half_grid=True, plot_contours=False,
);
plt.show()

#### contours=30

In [ ]:
%%timeit 
receivers, fw_fields = waveprop.run_waveprop(
    src, rec, model, absorbing_boundaries, 
    plot_wavefield=True, plot_wavefield_every=40,
    store_forward_fields=True,
    verbose=True,
    half_grid=False, plot_contours=True,
    levels=30,
);
plt.show()

#### contours=50

In [ ]:
%%timeit 
receivers, fw_fields = waveprop.run_waveprop(
    src, rec, model, absorbing_boundaries, 
    plot_wavefield=True, plot_wavefield_every=40,
    store_forward_fields=True,
    verbose=True,
    half_grid=False, plot_contours=True,
    levels=50,
);
plt.show();

#### Contours=30; half grid

In [ ]:
%%timeit 
receivers, fw_fields = waveprop.run_waveprop(
    src, rec, model, absorbing_boundaries, 
    plot_wavefield=True, plot_wavefield_every=40,
    store_forward_fields=True,
    verbose=True,
    half_grid=True,
    plot_contours=True, levels=30,
);
plt.show()